In [ ]:
# decision_engine.ipynb

# 1. Setup
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent / "src"))

from business_rules import RuleEngine, LLMExplainer, FinalDecisionOutput
from business_rules.data_generator import FraudDataGenerator
import pandas as pd

# 2. Generate synthetic data
generator = FraudDataGenerator(fraud_ratio=0.2)
df = generator.generate_dataset(n=50)
df.to_csv("data/sample_transactions.csv", index=False)

# 3. Initialize components
engine = RuleEngine("../config/rules_v1.yaml")
explainer = LLMExplainer()  # Uses ANTHROPIC_API_KEY env var

# 4. Process transactions
records = df.to_dict('records')
rule_results = engine.evaluate_batch(records)
explanations = explainer.generate_batch(records, rule_results)

# 5. Combine results
outputs = []
for record, rule_result, explanation in zip(records, rule_results, explanations):
    output = FinalDecisionOutput(
        transaction_id=record['transaction_id'],
        risk_score=rule_result.risk_score,
        decision=rule_result.decision,
        rule_matched=rule_result.matched_rule_name,
        rule_reason=rule_result.rule_reason,
        llm_explanation=explanation.human_readable_explanation,
        confidence=explanation.confidence,
        needs_human_review=explanation.needs_human_review,
        clarifying_questions=explanation.clarifying_questions
    )
    outputs.append(output.model_dump())

results_df = pd.DataFrame(outputs)

# 6. Display results
print(f"Total transactions: {len(results_df)}")
print(f"Flagged for human review: {results_df['needs_human_review'].sum()}")
print(f"\nDecision distribution:")
print(results_df['decision'].value_counts())

# 7. Show items needing review
review_items = results_df[results_df['needs_human_review'] == True]
for _, row in review_items.iterrows():
    print(f"\n--- Transaction {row['transaction_id']} ---")
    print(f"Decision: {row['decision']} (Score: {row['risk_score']})")
    print(f"Explanation: {row['llm_explanation']}")
    print(f"Questions: {row['clarifying_questions']}")